# Advanced Lane Finding: Video Processing Pipeline

---

## Importing Libraries

In [20]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
import pickle
%matplotlib inline
import matplotlib.image as mpimg

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Class Definitions 

### Line( ), Lane( )

In [21]:
# Defined a class to receive the characteristics of each line detection

class Line():
    def __init__(self):
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None
        # x values of the previous high-confidence fit of the line
        self.previousx = None
        
        #polynomial coefficients for the previous fit
        self.previous_fit = None
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None   
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        
        #distance in meters of vehicle center from the line
        #self.line_base_pos = None    ## Unused for sanity check, for offset we defined new variable
        
        #difference in fit coefficients between last and new fits
        #self.diffs = np.array([0,0,0], dtype='float')  ## can be used for sanity check
        
        
        #x values for detected line pixels 
        #self.allx = None  
        #y values for detected line pixels
        #self.ally = None 
        ## can be used to fit a line (better, since averaging is linear - but fit is quadratic)
    
# Defined a class to receive the characteristics of each lane detection    
    
class Lane:
    def __init__(self):
        self.detected = False
        self.sanity = False
        self.offset = 0
        self.radius_of_curvature = 0
    

## Various Function Definitions for Pipeline
------

### Correcting for Image Distortion: 

Camera Matrix and Distortion co-efficients have been derived from a set of test chess-board images

In [22]:
# Here image input is a colour one. Check if gray required (modify shape argument)

def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    return undist

### Colour and Gradient Thresholding:

Functions defined for sobel_x, sobel_y, sobel_xy and directional gradients

As for Colour spaces, Hue and Saturation thresholds (HLS Space) and Red threshold (RGB Space)

In [23]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Calculate directional gradient and apply threshold
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
 
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude and apply threshold
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return dir_binary

def sat_thresh(img,s_thresh=(90, 255)):

    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    return s_binary

def hue_thresh(img,h_thresh=(15, 100)):

    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    
    # Threshold color channel
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    
    return h_binary

def red_thresh(img,r_thresh=(200, 255)):

    r_channel = img[:,:,0]
    
    # Threshold color channel
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh[0]) & (r_channel <= r_thresh[1])] = 1
    
    return r_binary

### Combining different thresholding functions

In order to optimize the output binary image, a combination of the various functions defined above is taken

In [24]:
def threshold_combined(undist):
    
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements
    
    ## higher value, means additional pixels to left and right over which gradient is taken

    # Apply each of the thresholding functions

    ## Gradient_thresholding

    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(20, 100))  ##(10,150)
    grady = abs_sobel_thresh(undist, orient='y', sobel_kernel=ksize, thresh=(20, 100))  ##(10,150)
    mag_binary = mag_thresh(undist, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(undist, sobel_kernel=ksize, thresh=(0.7, 1.3))

    ## Colour_thresholding
    s_binary= sat_thresh(undist, s_thresh=(170,255))  ##changed from (90,255)
    r_binary= red_thresh(undist, r_thresh=(230,255))
    h_binary= hue_thresh(undist, h_thresh=(15,100))

    combined = np.zeros_like(dir_binary)

    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | ((s_binary==1) & (h_binary==1)) | r_binary==1] = 1
    
    # first two conditions identify crucial curve information
    # S & H identify coloured lines and work well despite of shadows and change in lighting
    # R is the best to identify white lines
    
    return combined

### Perspective Transform

Transform matrix M is derived from a test image where shape identification was simple

Vehicle View -> Birds-Eye View

In [25]:
def persp_transform(img,M,img_size):
    
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped

### Some Additional Helper Functions

In [26]:
def find_x_value (fit, y_eval):
    
    x_value = fit[0]*(y_eval**2) + fit[1]*y_eval + fit[2]
    
    return x_value

def pixel2metre (left_fit_p, right_fit_p):
    
    # Converts polynomial co-efficients from pixel to meter units
    
    # Define conversions in x and y from pixels space to meters
    my = 30/720 # meters per pixel in y dimension
    mx = 3.7/680 # meters per pixel in x dimension (680 for us as offset was 300)
    
    left_fit_m = np.zeros_like(left_fit_p)  #zeroes_like[left_fit_p] ??
    right_fit_m = np.zeros_like(right_fit_p)
    
    left_fit_m[0] = left_fit_p[0]* (mx / (my ** 2)) 
    left_fit_m[1] = left_fit_p[1]* (mx/my)
    left_fit_m[2] = left_fit_p[2]* mx
    
    right_fit_m[0] = right_fit_p[0]* (mx / (my ** 2)) 
    right_fit_m[1] = right_fit_p[1]* (mx/my)
    right_fit_m[2] = right_fit_p[2]* mx
    
    return left_fit_m, right_fit_m

### Fitting Curves: Finding Lane lines

#### Method 1: Sliding Windows

In [27]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    #out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty


def fit_polynomial(binary_warped):

    
    # Find our lane pixels first
    leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    #####AGAIN here error shouldnt be caused if we're using smoothing!!!!    

    return left_fitx, right_fitx, left_fit, right_fit

#### Method 2: Search Around Polynomial

In [28]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    
    #try:
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    
    #try:
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    #except:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
     #   print('The function failed to fit a line!')
      #  left_fitx = 1*ploty**2 + 1*ploty
       # right_fitx = 1*ploty**2 + 1*ploty
    
    return left_fitx, right_fitx, left_fit, right_fit

def search_around_poly(binary_warped, left_fit, right_fit):
    
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    return left_fitx, right_fitx, left_fit, right_fit

### Radius of curvature and Vehicle offset 

Calculates the Radius of Curvature of the two lane lines and positional offset of vehicle center from lane center

In [29]:
def measure_curvature_real(img_size, left_fit_m, right_fit_m):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    ploty = np.linspace(0, img_size[1]-1, img_size[1])
    
    # Define conversions in x and y from pixels space to meters
    my = 30/720 # meters per pixel in y dimension
    mx = 3.7/680 # meters per pixel in x dimension (680 for us as offset was 300)
    
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image 
    # i.e position where the car is atm
    y_eval = np.max(ploty)
    
    left_lane = find_x_value(left_fit_m, y_eval*my) 
    right_lane = find_x_value(right_fit_m, y_eval*my)
    
    lane_center = (left_lane + right_lane)/2
    frame_center = mx*640
    offset = lane_center - frame_center
    
    # Implement the calculation of R_curve (radius of curvature) 
    left_curverad = int(((1 + (2*left_fit_m[0]*y_eval*my + left_fit_m[1])**2)**1.5) / np.absolute(2*left_fit_m[0]))
    right_curverad = int(((1 + (2*right_fit_m[0]*y_eval*my + right_fit_m[1])**2)**1.5) / np.absolute(2*right_fit_m[0]))
    
    return left_curverad, right_curverad, offset

### Smoothing:

Adds the curve of current frame to list of previous frame curves and finds a weighted average

In [30]:
def append_average(left_fitx, left_lane, right_fitx, right_lane, img_size):
    
    ploty = np.linspace(0, img_size[1]-1, img_size[1])
    
    
    ## for left line 
    
    
    left_lane.recent_xfitted.append(left_fitx)
    
    if (len(left_lane.recent_xfitted) > 5):    ## n = 5 is maximum number of curves to hold
        
        del left_lane.recent_xfitted[0]        ## deletes oldest curve
        
    weights = np.arange(1, len(left_lane.recent_xfitted) + 1)    ## List of weights = (1,2,3,4,5)
        
    left_lane.bestx = np.average(left_lane.recent_xfitted, axis = 0, weights = weights)
        
    left_lane.best_fit = np.polyfit(ploty, left_lane.bestx, 2)
        
        
    ## for right line
    
    right_lane.recent_xfitted.append(right_fitx)
    
    if (len(right_lane.recent_xfitted) > 5):
        
        del right_lane.recent_xfitted[0]
        
    weights = np.arange(1, len(right_lane.recent_xfitted) + 1)
        
    right_lane.bestx = np.average(right_lane.recent_xfitted, axis = 0, weights = weights)
        
    right_lane.best_fit = np.polyfit(ploty, right_lane.bestx, 2)
    
    
    return


### Projecting Lanes:

Projects the lane measurements back down onto the road!

Displays curve information: Radius of Curvature and Offset

In [31]:
def draw_lanes(undist, warped, left_fitx, right_fitx, curverad, img_size, offset):
    
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, img_size) 
    
    # Creating Text and set font parameters
    #TextL = "Left  Curvature: " + str(format(left_curverad)) + " (m)"
    TextR = "Radius of Curvature: " + str(format(curverad))+ " (m)"
    fontScale=1
    thickness=2
    fontFace =cv2.FONT_HERSHEY_SIMPLEX 
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    # Using CV2 putText to write text into images
    #cv2.putText(result, TextL, (110,110), fontFace, fontScale,(255,255,255), thickness,  lineType = cv2.LINE_AA)
    cv2.putText(result, TextR, (110,60), fontFace, fontScale,(255,255,255), thickness,  lineType = cv2.LINE_AA)
    
    if offset < 0 :
        TextC = "Vehicle is " + str(format(np.absolute(offset), ".2f")) + " (m) to the right of center"
    elif offset > 0 :    
        TextC = "Vehicle is " + str(format(np.absolute(offset), ".2f")) + " (m) to the left of center"
    else:
        TextC = "Vehicle is at center"
    
    cv2.putText(result, TextC, (110,110), fontFace, fontScale,(255,255,255), thickness,  lineType = cv2.LINE_AA)
    
    return result

### Sanity Check

Performs multiple checks on curves fitted onto current frame in order to decide 
whether curve identification was effective

1. Difference between left and right curvatures
2. Lane width
3. Parallel lane lines

In [32]:
def sanity_check(left_fit_m, right_fit_m, left_curverad, right_curverad):
    
    check = False
    
    straight_lanes = False
    
    if ((left_curverad > 1000) & (right_curverad > 1000)):
        
        straight_lanes = True
    
    curve_gap = np.absolute(left_curverad - right_curverad)
    
    lane_gap_base = np.absolute(find_x_value(left_fit_m, 30) - find_x_value(right_fit_m, 30))
    
    #lane_gap_mid = np.absolute(find_x_value(left_fit_m, 15) - find_x_value(right_fit_m, 15))
    
    lane_gap_top = np.absolute(find_x_value(left_fit_m, 0) - find_x_value(right_fit_m, 0))
    
    if (((curve_gap < 400) | (straight_lanes))  & (np.absolute(lane_gap_base - 3.7) < 0.5) & (np.absolute(lane_gap_top - 3.7) < 0.7)):
        
    ## can put 0.5 and 0.7 in terms of percentage (15% of lane width, 20% of lane width)
    
        check = True
        
    return check  
        
    # lane width check for reality check
    # if the rad_curve are close enough and lane width is maintained, ~ lanes are parallel

## Choosing Pipeline:

### 1. Sliding Windows Only

In [33]:
def process_image_sliding(image):
    
    
    img_size = (image.shape[1],image.shape[0])
    
    undist = cal_undistort(image,mtx,dist)
    
    combined = threshold_combined(undist)
   
    warped = persp_transform(combined, M, img_size)
    
    left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(warped)
    
    left_fit_m, right_fit_m = pixel2metre(left_fit, right_fit)

    left_curverad, right_curverad, offset = measure_curvature_real(img_size, left_fit_m, right_fit_m)
    
    frame_check = sanity_check(left_fit_m, right_fit_m, left_curverad, right_curverad)
    
    global first_frame
    
    #print(frame_check)
    
    if (first_frame | frame_check):
    
        left_lane.bestx, right_lane.bestx = left_fitx, right_fitx
        
        left_lane.radius_of_curvature, right_lane.radius_of_curvature = left_curverad, right_curverad
        
        lane.radius_of_curvature = (left_lane.radius_of_curvature + right_lane.radius_of_curvature)/2
        
        lane.offset = offset
        
        left_lane.previous_fit, right_lane.previous_fit = left_fit, right_fit 
        
        first_frame = False   
       
    result = draw_lanes(undist, warped, left_lane.bestx, right_lane.bestx, lane.radius_of_curvature, img_size, lane.offset)
            
    return result
    
    #result = draw_lanes(undist, warped, ploty, left_fitx, right_fitx, left_curverad, right_curverad, img_size, offset)
    
    #print(np.absolute(left_curverad - right_curverad))
    
    #lane_gap_base = np.absolute(find_x_value(left_fit, 30) - find_x_value(right_fit, 30))
    
    #lane_gap_mid = np.absolute(find_x_value(left_fit, 0) - find_x_value(right_fit, 0))
    
    #print(np.absolute(lane_gap_base - 3.7))
    
    #print(np.absolute(lane_gap_mid - 3.7))
   

### 2. Look-Ahead Filter with Sliding Windows as reset option

In [34]:
def process_image_skipping(image):
    
    
    img_size = (image.shape[1],image.shape[0])
    
    undist = cal_undistort(image,mtx,dist)
    
    combined = threshold_combined(undist)
   
    warped = persp_transform(combined, M, img_size)
    
    global first_frame
    
    global frame_count
    
    if ( first_frame | (frame_count > 2)):
    
        left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(warped)
        
        #print('sliding')
        
        first_frame = False
        
        frame_count = 0
        
    else:
        
        try:
            
            left_fitx, right_fitx, left_fit, right_fit = search_around_poly(warped, left_lane.previous_fit, right_lane.previous_fit)
        
            #print('skipping')
            
        except TypeError: 
            
            try: 
                
                left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(warped)
                
                #print('sliding')
            
            except TypeError:
                
                pass
                
    left_fit_m, right_fit_m = pixel2metre(left_fit, right_fit)
    
    left_curverad, right_curverad, offset = measure_curvature_real(img_size, left_fit_m, right_fit_m)

    frame_check = sanity_check(left_fit_m, right_fit_m, left_curverad, right_curverad)
    
    #print(frame_check)

    if (frame_check):
        
        lane.detected = True

        left_lane.bestx, right_lane.bestx = left_fitx, right_fitx

        left_lane.radius_of_curvature, right_lane.radius_of_curvature = left_curverad, right_curverad
        
        lane.radius_of_curvature = (left_lane.radius_of_curvature + right_lane.radius_of_curvature)/2
        
        lane.offset = offset

        left_lane.previous_fit, right_lane.previous_fit = left_fit, right_fit

        frame_count = 0

    else:

        frame_count+=1
    
    
    result = draw_lanes(undist, warped, left_lane.bestx, right_lane.bestx, lane.radius_of_curvature, img_size, lane.offset)
    
    return result
    
   

### 3. Complete Pipeline: Sanity Check, Tracking, Look-Ahead Filter, Reset and Smoothing

In [35]:
def process_image_smoothing(image):
    
    
    img_size = (image.shape[1],image.shape[0])
    
    undist = cal_undistort(image,mtx,dist)
    
    combined = threshold_combined(undist)
   
    warped = persp_transform(combined, M, img_size)
    
    global frame_count
    
    if (lane.sanity == False | (frame_count > 2) | lane.detected == False):
    
        try:
            
            left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(warped)
            
            lane.detected = True
            
        except TypeError:
            
            lane.detected = False
        
        #print('sliding')
        
        frame_count = 0
        
    else:
        
        try:
            
            left_fitx, right_fitx, left_fit, right_fit = search_around_poly(warped, left_lane.previous_fit, right_lane.previous_fit)
        
            lane.detected = True
            
            #print('skipping')
            
        except TypeError: 
            
            try: 
                
                left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(warped)
                
                lane.detected = True
                
                #print('sliding')
            
            except TypeError:
                
                lane.detected = False 
                
    left_fit_m, right_fit_m = pixel2metre(left_fit, right_fit)
    
    left_curverad, right_curverad, offset = measure_curvature_real(img_size, left_fit_m, right_fit_m)
    

    frame_check = sanity_check(left_fit_m, right_fit_m, left_curverad, right_curverad)
    
    #print(frame_check)
    

    if (frame_check):
        
        lane.sanity = True
        
        left_lane.previousx, right_lane.previousx = left_fitx, right_fitx  # recording high-confidence result
        
        left_lane.previous_fit, right_lane.previous_fit = left_fit, right_fit # for use in search_poly (next iteration)
        
        append_average(left_fitx, left_lane, right_fitx, right_lane, img_size)
        
        frame_count = 0
        
    else:
        
        lane.sanity = False
        
        append_average(left_lane.previousx, left_lane, right_lane.previousx, right_lane, img_size)
        
        #instead of directly using previous best fit, increase the weight of previous frame fit and then average it
        
        frame_count+=1
    
    
    left_fit_r, right_fit_r = pixel2metre(left_lane.best_fit, right_lane.best_fit)
    
    left_lane.radius_of_curvature, right_lane.radius_of_curvature, lane.offset = measure_curvature_real(img_size, left_fit_r, right_fit_r)

    lane.radius_of_curvature = (left_lane.radius_of_curvature + right_lane.radius_of_curvature)/2

    # Didn't need current_fit as it is available in left_fit/right_fit in the given iteration
    # Hence, no need to store
        
    
    result = draw_lanes(undist, warped, left_lane.bestx, right_lane.bestx, lane.radius_of_curvature, img_size, lane.offset)
    
    return result
    
   

### Initializations and reading in data files

In [39]:
#Initializations

left_lane = Line()
right_lane = Line()
lane = Lane()

frame_count = 0
frame_check = False
first_frame = True

# Read in the saved camera matrix and distortion co-efficients
dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

persp_pickle = pickle.load( open( "persp_pickle.p", "rb" ) )
M = persp_pickle["M"]
Minv = persp_pickle["Minv"]

### make sure persp_pickle is latest version from calibration

## Running Pipeline on Video:

Fill the selected pipeline option (process_image_sliding/process_image_skipping/process_image_smoothing) as the argument for clip1.fl_image( )

In [40]:
write_output = 'output_videos/smoothing_pipeline.mp4'
#write_output = 'output_videos/skipping_pipeline.mp4'
#write_output = 'output_videos/sliding_pipeline.mp4'


## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(0,5)


clip1 = VideoFileClip("project_video.mp4")


white_clip = clip1.fl_image(process_image_smoothing) #NOTE: this function expects color images!!
#white_clip = clip1.fl_image(process_image_skipping)
#white_clip = clip1.fl_image(process_image_sliding)


%time white_clip.write_videofile(write_output, audio=False)


t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/smoothing_pipeline.mp4.
Moviepy - Writing video output_videos/smoothing_pipeline.mp4



Moviepy - Done !
Moviepy - video ready output_videos/smoothing_pipeline.mp4
CPU times: user 3min 46s, sys: 48.6 s, total: 4min 34s
Wall time: 3min 54s


In [41]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(write_output))